### Methods to implment Tiny Darknet model architecture

In [8]:
def preprocess_true_boxes(true_boxes, input_shape, anchors, num_classes):
    '''Preprocess true boxes to training input format
    Parameters
    ----------
    true_boxes: array, shape=(m, T, 5)
        Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
    input_shape: array-like, hw, multiples of 32
    anchors: array, shape=(N, 2), wh
    num_classes: integer
    Returns
    -------
    y_true: list of array, shape like yolo_outputs, xywh are reletive value
    '''
    assert (true_boxes[..., 4]<num_classes).all(), 'class id must be less than num_classes'
    num_layers = len(anchors)//3 # default setting
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]]

    true_boxes = np.array(true_boxes, dtype='float16')
    input_shape = np.array(input_shape, dtype='int16')
    boxes_xy = true_boxes[..., 0:2]
    boxes_wh = true_boxes[..., 2:4]

    m = true_boxes.shape[0]
    grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
    y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]

    # Expand dim to apply broadcasting.
    anchors = np.expand_dims(anchors, 0)
    anchor_maxes = anchors / 2.
    anchor_mins = -anchor_maxes
    valid_mask = boxes_wh[..., 0]>0

    for b in range(m):
        # Discard zero rows.
        wh = boxes_wh[b, valid_mask[b]]
        if len(wh)==0: continue
        # Expand dim to apply broadcasting.
        wh = np.expand_dims(wh, -2)
        box_maxes = wh / 2.
        box_mins = -box_maxes

        intersect_mins = np.maximum(box_mins, anchor_mins)
        intersect_maxes = np.minimum(box_maxes, anchor_maxes)
        intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
        box_area = wh[..., 0] * wh[..., 1]
        anchor_area = anchors[..., 0] * anchors[..., 1]
        iou = intersect_area / (box_area + anchor_area - intersect_area)

        # Find best anchor for each true box
        best_anchor = np.argmax(iou, axis=-1)

        for t, n in enumerate(best_anchor):
            for l in range(num_layers):
                if n in anchor_mask[l]:
                    i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
                    j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
                    k = anchor_mask[l].index(n)
                    c = true_boxes[b,t, 4].astype('int32')
                    y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
                    y_true[l][b, j, i, k, 4] = 1
                    y_true[l][b, j, i, k, 5+c] = 1

    return y_true


def box_iou(b1, b2):
    '''Return iou tensor
    Parameters
    ----------
    b1: tensor, shape=(i1,...,iN, 4), xywh
    b2: tensor, shape=(j, 4), xywh
    Returns
    -------
    iou: tensor, shape=(i1,...,iN, j)
    '''

    # Expand dim to apply broadcasting.
    b1 = K.expand_dims(b1, -2)
    b1_xy = b1[..., :2]
    b1_wh = b1[..., 2:4]
    b1_wh_half = b1_wh/2.
    b1_mins = b1_xy - b1_wh_half
    b1_maxes = b1_xy + b1_wh_half

    # Expand dim to apply broadcasting.
    b2 = K.expand_dims(b2, 0)
    b2_xy = b2[..., :2]
    b2_wh = b2[..., 2:4]
    b2_wh_half = b2_wh/2.
    b2_mins = b2_xy - b2_wh_half
    b2_maxes = b2_xy + b2_wh_half

    intersect_mins = K.maximum(b1_mins, b2_mins)
    intersect_maxes = K.minimum(b1_maxes, b2_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
    b1_area = b1_wh[..., 0] * b1_wh[..., 1]
    b2_area = b2_wh[..., 0] * b2_wh[..., 1]
    iou = intersect_area / (b1_area + b2_area - intersect_area)

    return iou


In [ ]:
def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss) #return a model with the y_preds and y_true with a specific loss

    return model

In [4]:
def make_csv(path ,train_txt, name):
    list_images = []
    list_labels = []
    list_anchors = []
    list_class  = []

    for file in tqdm(open(path+train_txt).readlines()):
        file = file.replace('\n','')
        file = file.replace('\r','')
        img_path = path+file[2:]
        img_path_copy = img_path
    
        #target_image = cv2.imread(img_path)
        img_path=img_path.replace(".jpg",".txt")
        img_path= img_path.replace(".png",".txt")
        lines = open(img_path).readlines()
        anchors =  0
        string_label =''
        for line in lines:
            anchors = anchors+1
            line = line.replace("\n","")
            string_label = string_label+"!"+line  # all labels are seperated by !
            
        if anchors!=0:
            list_labels.append(string_label)
            list_anchors.append(anchors)
            list_images.append(img_path_copy)  #append the path of the image
            list_class.append("Face")


    images = np.array(list_images)
    labels = np.array(list_labels)
    anchors = np.array(list_anchors)
    classes = np.array(list_class)
    training  = pd.DataFrame({'target image path':images , 'labels': labels , 'class': classes , 'anchors': anchors})
    training.to_csv(name , index = False)   #write the results to an csv file   

### CSV Generator

In [1]:
#image_from_csv Generator
import sklearn
def csv_image_generator(csv_file , batch , ls , num_classes, anchors , mode = 'train' , data_augmentation = None): #these are the default values
    csv_file = pd.read_csv(csv_file)
    csv_file = sklearn.utils.shuffle(csv_file)
    csv_file.reset_index(inplace=True, drop=True)
    max_boxes = 20
    counter = 1
    max_rows = len(csv_file.index)
    while True: #a generator loops infinitely,hencewe need to use steps per epoch to determine when a batch starts n stop
        # initialize our batches of images and labels
        input_shape = data_augmentation
        images = []
        batch_boxes = []

        while len(images) < batch: #loop until batch size is met
            box_array = np.zeros((max_boxes,5))
            row = csv_file.iloc[counter, 0:4]
            image = cv2.imread(row['target image path'])
            # if the data augmentation 
            if data_augmentation is not None: #aug has the (width,height) shape of the image
                image = cv2.resize(image,data_augmentation) #resize image according to (width,height) either 416x416 or 224x224
                
            string_label = row['labels'] #get the whole stringlabel
            
            if not (string_label=="" or string_label=="nan"):
                boxes = string_label.split('!')
                boxes = np.array(boxes)
                num_anchors = row['anchors'] #number of bounding boxes per image
                empty_list = []
                for box in boxes:
                    if not box=='':
                        labels = box.split(" ")
                        labels = np.array(labels)
                        class_= labels[0] #get the class id
                        box_data = [labels[1], labels[2], labels[3],labels[4],int(labels[0])] #set class id at position 4
                        empty_list.append(box_data)
                        #ls.transform(class_) activate when class training starts

                all_boxes_per_image = np.array(empty_list)
                if len(all_boxes_per_image)>max_boxes: all_boxes_per_image = all_boxes_per_image[:max_boxes]
                box_array[:len(all_boxes_per_image)] = all_boxes_per_image

                # check to see if the line is empty, indicating we have
                # reached the end of the file
                if counter == max_rows-1 :
                    counter = 1 #go back to the beginning of the file
                    #print('Start form the beginning of the dataset')

                    if mode == "eval": #for evaluation mode break if we reached the end of the file
                        break
                #update batch list
                images.append(image)
                #print(box_array.shape)
                batch_boxes.append(box_array)
                
            counter = (counter+1)%max_rows
            
            
        #End of batch size loop
        batch_boxes = np.array(batch_boxes)
        #print(batch_boxes.shape)
        y_true = preprocess_true_boxes(batch_boxes ,input_shape, anchors, num_classes)
        # yield the batch to the calling function /the generator returns a batch everytime
        yield [np.array(images), *y_true], np.zeros(batch) #keep yielding the function to the generator while its true
      
    #end of while loop

In [4]:
def plot_performance(H,N):
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")  #accuracy only applies when we have different classes
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy on Dataset") #consider doing an IoU vs number of epochs for every batch
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("plot.png")

In [2]:
def csv_image_generator_wrapper(csv_file , batch , ls , num_classes, anchors , mode , data_augmentation):
    if batch<=0: return None
    return csv_image_generator(csv_file , batch , ls , num_classes, anchors ,mode , data_augmentation)


In [7]:
def tiny_yolo_body(inputs, num_anchors, num_classes):
    '''Create Tiny YOLO_v3 model CNN body in keras.'''
    x1 = compose(
            DarknetConv2D_BN_Leaky(16, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(32, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(64, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(128, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(256, (3,3)))(inputs)
    x2 = compose(
            MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),
            DarknetConv2D_BN_Leaky(512, (3,3)),
            MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'),
            DarknetConv2D_BN_Leaky(1024, (3,3)),
            DarknetConv2D_BN_Leaky(256, (1,1)))(x1)
    y1 = compose(
            DarknetConv2D_BN_Leaky(512, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))(x2)

    x2 = compose(
            DarknetConv2D_BN_Leaky(128, (1,1)),
            UpSampling2D(2))(x2)
    y2 = compose(
            Concatenate(),
            DarknetConv2D_BN_Leaky(256, (3,3)),
            DarknetConv2D(num_anchors*(num_classes+5), (1,1)))([x2,x1])

    return Model(inputs, [y1,y2])

In [8]:
def yolo_head(feats, anchors, num_classes, input_shape, calc_loss=False):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]),
        [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]),
        [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    if calc_loss == True:
        return grid, feats, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs

In [9]:
def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes


def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores


def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)

    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)

    return boxes_, scores_, classes_

### Training    

In [9]:
#Input images need to be OF SPECIFIC SIZE 224X224
#import all necessary libraries
import tensorflow.keras
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
from keras import backend as K
import numpy as np
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from yolo3.model import yolo_body, tiny_yolo_body, yolo_loss
from sklearn.preprocessing import LabelBinarizer, LabelEncoder


#check keras gpu is running
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
K.set_session(sess)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.keras.backend.clear_session()


train_txt = "dataset/fddb/FDDB-folds/annotations_darknet/train.txt"
test_txt = "dataset/fddb/FDDB-folds/annotations_darknet/test.txt"
path = "D:/Desktop/Facial-Recognition/Face-Detection/" #path to Face Detection folder

make_csv(path,train_txt, "training.csv") #this creates a training.csv file with everything needed for training
make_csv(path , test_txt, "testing.csv")  #this created the testing csv file
training_data = pd.read_csv("training.csv")       
#image =  cv2.imread(training_data.iat[0,0])
#cv2.imshow("test", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#print(image.shape)



#Set trainig parameters
TRAIN_CSV = "training.csv"
TEST_CSV = "testing.csv"
INPUT_SHAPE = (416,416)
ANCHORS = np.array([10,14,  23,27,  37,58,  81,82,  135,169,  344,319]).reshape(-1, 2) #the anchors go by pairs
EPOCHS = 3
BATCH = 5
NUM_CLASSES = 1



model = create_tiny_model(INPUT_SHAPE, ANCHORS, NUM_CLASSES, load_pretrained= False, freeze_body=2, weights_path='keras-yolo3/model_data/tiny_yolo_weights.h5')
model_checkpoint = ModelCheckpoint("model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
# Count the samples/#of rows for testing and training dataset
NUM_TRAIN_IMAGES = (pd.read_csv(TRAIN_CSV)).shape[0]
NUM_TEST_IMAGES = (pd.read_csv(TEST_CSV)).shape[0]


object_encoder = LabelBinarizer()
object_encoder.fit(["Face", "Nada"])
class_encoder = LabelEncoder()
class_encoder.fit(["Face", "Nada"])

#testclasses = lb.transform(test_classes)


# initialize both the training and testing image generators
train_gen = csv_image_generator_wrapper(TRAIN_CSV, BATCH, object_encoder,NUM_CLASSES, ANCHORS, "train", INPUT_SHAPE)
test_gen = csv_image_generator_wrapper(TEST_CSV, BATCH, object_encoder ,NUM_CLASSES,ANCHORS, "eval", INPUT_SHAPE)

# initialize our Keras model and compile it

opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)
model.compile(loss={'yolo_loss': lambda y_true, y_pred: y_pred}, optimizer=opt)

# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(train_gen, steps_per_epoch=NUM_TRAIN_IMAGES // BATCH, 
                        validation_data=test_gen,validation_steps=NUM_TEST_IMAGES // BATCH,
                        epochs=EPOCHS,initial_epoch = 0 ,callbacks = [model_checkpoint])



# make predictions on the testing/validation images
predIdxs = model.predict_generator(test_gen,steps=(NUM_TEST_IMAGES // BATCH) + 1) #should return a 3D tensor with (samples, bounding_boxes , bounding_box offsets)



# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testLabels.argmax(axis=1), predIdxs,target_names=lb.classes_))

# plot the training loss and accuracy
plot_performance(H ,EPOCHS)




 33%|█████████████████████████▋                                                    | 234/711 [00:00<00:00, 2182.10it/s]

Num GPUs Available:  1


100%|████████████████████████████████████████████████████████████████████████████| 2134/2134 [00:00<00:00, 5059.07it/s]


Create Tiny YOLOv3 model with 6 anchors and 1 classes.
[INFO] training w/ generator...
Epoch 1/3


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:3335) ]]
	 [[loss/add_12/_383]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:3335) ]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node conv2d_1/convolution:
 input_1 (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:507)	
 conv2d_1/kernel

In [ ]:
#Calculate the number of MACs operations of your model (number of flops)
flops = tf.profiler.profile(graph,\
     options=tf.profiler.ProfileOptionBuilder.float_operation())
print('FLOP = ', flops.total_float_ops)

In [11]:
#Load model
from keras.models import load_model
model1 = load_model('model.h5')
image = cv2.imread('face.jpg')
image = cv2.resize(image, (416,416))
model1.predict(image)

ValueError: No model found in config file.

In [10]:
import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np

'''Make the training and testing csv file'''
train_txt = "dataset/fddb/FDDB-folds/annotations_darknet/train.txt"
test_txt = "dataset/fddb/FDDB-folds/annotations_darknet/test.txt"
path = "D:/Desktop/Facial-Recognition/Face-Detection/" #path to Face Detection folder

make_csv(path,train_txt, "training.csv") #this creates a training.csv file with everything needed for training
make_csv(path , test_txt, "testing.csv")  #this created the testing csv file

100%|████████████████████████████████████████████████████████████████████████████| 2134/2134 [00:00<00:00, 4059.06it/s]


In [11]:
'''Convert the yolo output format to the absolute coordinates and place them in the train text file'''
text_file = "train_faces.txt"
training_csv = pd.read_csv("training.csv")
f = open(text_file,'a') # a stands for append and w for write

for i in range (1, training_csv.shape[0]):
    '''For every row of the csv file extract the image file path'''
    path = training_csv.iat[i,0]
    labels = training_csv.iat[i,1]
    inputs = labels.split('!')
    image = cv2.imread(path)
    image_width = image.shape[1]
    image_height = image.shape[0]
    inputs.remove(inputs[0])
    f.write(path)
    for input_ in inputs:
        '''For every box in the specific image extract the box offsets
        and calculate abs coordinates'''
        string =str(input_)
        bb= string.split()
        input_x = float(bb[1])
        input_y = float(bb[2])
        input_w =float(bb[3])
        input_h =float(bb[4])
        class_ = int(bb[0])
        absx_center = float(input_x *image_width)
        absy_center = float(input_y *image_height)

        abs_boxh = float(input_h *image_height)
        abs_boxw = float(input_w *image_width)

        xmin= int(absx_center - abs_boxw//2)
        xmax= int(absx_center + abs_boxw/2)
        ymin = int(absy_center - abs_boxh/2)
        ymax =int(absy_center + abs_boxh/2)

        ''' Check if the conversion was completed succesfully
        cv2.rectangle(image, (xmin,ymin),(xmax,ymax), (0,255,255), 3)
        cv2.imshow('hello' , image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        '''
        #add xmin,xmax,ymin,ymax in the file--> image_path box1 box2 ... boxN
        box = ','.join([str(x) for x in [xmin,ymin,xmax,ymax,class_]])
        f.write(' '+box)
    f.write('\n') #change line after you have written all boxes
f.close()

### Train Model

In [16]:
'Import all librares needed to run the train.py file'
#import all necessary libraries
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2
"""
Retrain the YOLO model for your own dataset.
"""

import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
import tensorflow as tf




def _main():
    annotation_path = 'train_faces.txt'
    log_dir = './'
    classes_path = 'yolo3/model_data/faces.txt'
    anchors_path = 'yolo3/model_data/tiny_yolo_anchors.txt'
    class_names = get_classes(classes_path)
    num_classes = len(class_names)
    anchors = get_anchors(anchors_path)
    input_shape = (416,416) # multiple of 32, hw
    is_tiny_version = len(anchors)==6 # default setting
    if is_tiny_version:
        model = create_tiny_model(input_shape, anchors, num_classes,
            freeze_body=0, weights_path='yolo3/model_data/tiny_yolo_weights.h5')
    #logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val

    # Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 1
        model.save('tiny_yolo.h5')
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=10,
                initial_epoch=0, callbacks = [checkpoint])
        model.save_weights(log_dir + 'trained_weights_stage_1.h5')

    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 12 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[ checkpoint, reduce_lr, early_stopping])
        model.save_weights(log_dir + 'trained_weights_final.h5')

    # Further training if needed.


def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)




def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

if __name__ == '__main__':
    _main()


Create Tiny YOLOv3 model with 6 anchors and 2 classes.


D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\engine\topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_10 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\engine\topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_10 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\engine\topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_13 due to mismatch in shape ((1, 1, 256, 21) vs (255, 256, 1, 1)).
  weight_values[i].shape))
D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\engine\topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_13 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))


Load weights yolo3/model_data/tiny_yolo_weights.h5.
Train on 5620 samples, val on 624 samples, with batch size 1.
Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:3335) ]]
	 [[loss/add_12/_879]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:3335) ]]
0 successful operations.
0 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node conv2d_1/convolution:
 conv2d_1/kernel/read (defined at D:\Conda Installation\envs\gpu_python3.6\lib\site-packages\keras\backend\tensorflow_backend.py:395)	
 in